# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## 1. Изучение данных из файла

In [1]:
import pandas as pd
bank_data = pd.read_csv('data.csv')
display(bank_data.head(5))
bank_data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В таблице 12 колонок, типы данных - float64, int64, object

В названиях столбцов нарушений нет.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

В колонке days_employed есть отрицательные значения.

Согласно документации к данным:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

## 2. Предобработка данных

### Обработка пропусков

In [2]:
# Заменяю пропуски в столбце на медианное значение
print('Пропуски до (в столбце Days employed):', bank_data['days_employed'].isna().sum())
bank_data.loc[bank_data['days_employed'].isna(), 'days_employed'] = bank_data['days_employed'].median()
print('Пропуски после (в столбце Days employed):', bank_data['days_employed'].isna().sum())

Пропуски до (в столбце Days employed): 2174
Пропуски после (в столбце Days employed): 0


In [3]:
# Заменяю пропуски в столбце на медианное значение
print('Пропуски до (в столбце Total income):', bank_data['total_income'].isna().sum())
bank_data.loc[bank_data['total_income'].isna(), 'total_income'] = bank_data['total_income'].median()
print('Пропуски после (в столбце Total income):', bank_data['days_employed'].isna().sum())

Пропуски до (в столбце Total income): 2174
Пропуски после (в столбце Total income): 0


In [4]:
# Показывает значение -1 и 20
print(bank_data['children'].unique())  
# Показывает значение XNA
print(bank_data['gender'].unique()) 
# Нахожу самый рапространенный пол в таблице
print(bank_data['gender'].value_counts()) 
# Заменяю значение
bank_data['children'] = bank_data['children'].replace(-1, 0) 
# Заменяю значение
bank_data['children'] = bank_data['children'].replace(20, 2) 
# Заменяю значение
bank_data['gender'] = bank_data['gender'].replace('XNA', 'F') 
print(bank_data['children'].unique())
print(bank_data['gender'].unique())

[ 1  0  3  2 -1  4 20  5]
['F' 'M' 'XNA']
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
[1 0 3 2 4 5]
['F' 'M']


In [5]:
# Количество несовершеннолетних
print(bank_data[bank_data['dob_years'] < 18]['dob_years'].count()) 
# Замена несовершеннолетних
bank_data.loc[bank_data['dob_years'] < 18, 'dob_years'] = int(bank_data['dob_years'].median()) 
# Количество несовершеннолетних после замены
print(bank_data[bank_data['dob_years'] < 18]['dob_years'].count()) 

101
0


**Вывод**

1) Нашел пропущенные значения в столбце days_employed и в столбце total_income. Также в толбце children были странные значения
-1 и 20, заменил их на 0 и 2. В столбце gender, было значение XNA, его заменил на самый распространенный пол в таблице. Также обнаружил несовершеннолетних (возраст меньше 18 в колонке dob_years), заменил на медиану. 

2) Количество пропусков 2174, если пропуск в days_employed, то он есть и в total_income, возможно это связано с неофициальным трудоустройством или чем-то подобным. По типу пропуска, думаю это тип MAR (Missing At Random).

3) Пропуски заполнял медианной. 

### Замена типа данных

In [6]:
# Перевожу столбец в целочисленный формат
bank_data['total_income'] = bank_data['total_income'].astype('int') 
# Меняю отрицательные значения на положительные
bank_data['days_employed'] = abs(bank_data['days_employed']) 
# Перевожу стаж из часов в дни
bank_data.loc[bank_data['days_employed'] >30000, 'days_employed'] = bank_data.loc[bank_data['days_employed'] >30000, 'days_employed'] / 24
# Перевожу столбец в целочисленный формат
bank_data['days_employed'] = bank_data['days_employed'].astype('int')  
# Приводим к нижнему регистру
bank_data['education'] = bank_data['education'].str.lower() 
# Приводим к нижнему регистру
bank_data['family_status'] = bank_data['family_status'].str.lower() 
display(bank_data.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Для перевода из вещественного формата в целочисленный использовал метод astype(), потому что данный метод лучше подходит для решения этой задачи. Метод to_numeric() превращает значения столбца в числовой тип float64 (вещественное число).  

### Обработка дубликатов

In [7]:
print('Количество дубликатов до:', bank_data.duplicated().sum()) # Нахожу дубликаты
bank_data = bank_data.drop_duplicates().reset_index(drop = True) # Удаляю дубликаты 
print('Количество дубликатов после:', bank_data.duplicated().sum()) # Проверяю дубликаты  

Количество дубликатов до: 72
Количество дубликатов после: 0


**Вывод**

1) Для данной таблицы выбрал метод duplicated(), потому что этот метод обрабатывает всю таблицу, а не столбец как метод value_counts(). В таблице много повторяющихся значений и нет id для каждого пользователя. Метод уникальных значений здесь не подойдет. 

2) Возможная причина пояления дубликатов - неправильное соединение баз. База была не одна, при соединении в общую таблицу продублировались некоторые клиенты.   

### Лемматизация

In [8]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
# Получил уникальные значения purpose, склеил их в одну строку с помощью join и лемматизировал
lemmas = m.lemmatize(' '.join(bank_data['purpose'].unique()))
# Посчитал частоту появления слов.
Counter(lemmas).most_common()

# Основные категории: недвижимость, жилье, свадьба, автомобиль, образование    

# Написал функцию для apply с лемматизацией
def purpose_list_function(text): 
    lemmas = m.lemmatize(text) 
    if 'недвижимость' in lemmas:
        return 'недвижимость'
    elif 'жилье' in lemmas:
        return 'недвижимость'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'автомобиль' in lemmas:
        return 'авто'
    elif 'образование' in lemmas:
        return 'образование'
    else:
        return 'прочее'

bank_data['purpose_list'] = bank_data['purpose'].apply(purpose_list_function)
display(bank_data.head(10))
print(bank_data['purpose_list'].value_counts())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_list
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,авто
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


недвижимость    10811
авто             4306
образование      4013
свадьба          2323
Name: purpose_list, dtype: int64


**Вывод**

Импортировал необходимые библиотеки. Получил уникальные значения purpose, склеил их в одну строку с помощью join и лемматизировал. Посчитал частоту появления слов при помощи Counter. Также использовал метод value_counts, чтобы подробнее изучить цели кредита. Основными категориями оказались недвижимость или жилье, свадьбы, автомобили, образование. В функции указал раздел прочее, но он не понадобился. Все цели кредита попали под основные категории.    

### Категоризация данных

In [9]:
# Классификация по типу
education_dict = bank_data[['education','education_id']].drop_duplicates().set_index('education_id') 
family_dict = bank_data[['family_status','family_status_id']].drop_duplicates().set_index('family_status_id') 
print(education_dict)
print(family_dict)


                        education
education_id                     
0                          высшее
1                         среднее
2             неоконченное высшее
3                       начальное
4                  ученая степень
                          family_status
family_status_id                       
0                       женат / замужем
1                      гражданский брак
2                        вдовец / вдова
3                             в разводе
4                 не женат / не замужем


In [10]:
# Классификация по возрастным группам
def dob_years_function(age):
    if age < 18:
        return 'дети'
    if age <= 64:
        return 'взрослые'
    return 'пенсионеры' 

bank_data['dob_years_group'] = bank_data['dob_years'].apply(dob_years_function) 
print(bank_data['dob_years_group'].value_counts())

взрослые      20558
пенсионеры      895
Name: dob_years_group, dtype: int64


In [11]:
# Классификация по доходам
def total_income_function(money):
    if money < 100000:
        return 'низкий доход'
    if 100000 <= money < 300000:
        return 'средний доход'
    return 'высокий доход'

bank_data['total_income_group'] = bank_data['total_income'].apply(total_income_function) 
print(bank_data['total_income_group'].value_counts())

средний доход    15508
низкий доход      4463
высокий доход     1482
Name: total_income_group, dtype: int64


In [12]:
# Классификация по наличию детей
def children_function(child):
    if child > 0:
        return 'есть дети'
    return 'нет детей'

**Вывод**

Применил классификацию по типу для колонок education и family_status для того, чтобы оптимально хранить информацию о категориях. Использовал классификацию по возрастным группам для колонки dob_years, для выполнения задач по этому кейсу - это было не нужно, но для проверки и удобства сделал и такую классификацию. Для колонки total_income сделал классификацию по доходам (низкий, средний, высокий), чтобы проще было анализировать базу и ответить на вопрос: 'Есть ли зависимость между уровнем дохода и возвратом кредита в срок?'. Также сделал классификацию по наличию детей для удобства.


## 3. Исследовательский анализ данных

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [13]:
data_children_debt = bank_data.pivot_table(index=['children'], columns = 'debt', values = 'total_income', aggfunc = 'count')
data_children_debt.columns = ['no_debt', 'debt']
data_children_debt['%'] = (data_children_debt['debt'] / (data_children_debt['no_debt'] + data_children_debt['debt'])) * 100
data_children_debt = data_children_debt.fillna(0)
display(data_children_debt)

,no_debt,debt,%
children,,,
0,13073.0,1064.0,7.526349
1,4364.0,444.0,9.234609
2,1926.0,202.0,9.492481
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,0.0,0.000000


**Вывод**

В анализируемой таблице мы видим незначительное превышение процентных пунктов невозвратов у группы людей с детьми, чем в группе людей без детей. Отклонения несущественные, недостаточно факторов для более точного анализа, нужна большая выборка. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [14]:
data_family_debt = bank_data.pivot_table(index=['family_status'], columns = 'debt', values = 'total_income', aggfunc = 'count')
data_family_debt.columns = ['no_debt', 'debt']
data_family_debt['%'] = (data_family_debt['debt'] / (data_family_debt['no_debt'] + data_family_debt['debt'])) * 100
display(data_family_debt)

,no_debt,debt,%
family_status,,,
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3762,388,9.349398
женат / замужем,11408,931,7.545182
не женат / не замужем,2536,274,9.750890


**Вывод**

Данные категории граждан можно объединить в три группы. Первая группа - гражданский брак и не женат / не замужем. Вторая группа - женат / замужем. Третья группа - вдовец / вдова и в разводе. По результатам анализа у первой группы процентных пунктов невозвратов больше всего, далее идет вторая группа, наименьший уровень процентных пунктов у третьей группы.    

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [15]:
data_income_debt = bank_data.pivot_table(index=['total_income_group'], columns = 'debt', values = 'total_income', aggfunc = 'count')
data_income_debt.columns = ['no_debt', 'debt']
data_income_debt['%'] = (data_income_debt['debt'] / (data_income_debt['no_debt'] + data_income_debt['debt'])) * 100
display(data_income_debt)

,no_debt,debt,%
total_income_group,,,
высокий доход,1376,106,7.152497
низкий доход,4109,354,7.931884
средний доход,14227,1281,8.260253


**Вывод**

В данной таблице мы видим, что наибольший уровень процентных пунктов у людей со средним доходом, однако у людей с низким доходом уровень процентных пунктов невозвратов тоже высок. Это может быть связано с тем, что обладатели среднего дохода чаще берут кредит, чем люди с низким или высоким доходами. Люди с высокими доходами показали лучший результат.   

- Как разные цели кредита влияют на его возврат в срок?

In [16]:
data_purpose_debt = bank_data.pivot_table(index=['purpose_list'], columns = 'debt', values = 'total_income', aggfunc = 'count')
data_purpose_debt.columns = ['no_debt', 'debt']
data_purpose_debt['%'] = (data_purpose_debt['debt'] / (data_purpose_debt['no_debt'] + data_purpose_debt['debt'])) * 100
display(data_purpose_debt)

,no_debt,debt,%
purpose_list,,,
авто,3903,403,9.359034
недвижимость,10029,782,7.233373
образование,3643,370,9.220035
свадьба,2137,186,8.006888


**Вывод**

Из данных в приведённой таблице мы видим, что по кредитам на недвижимость самый маленький процент просрочки кредитов. Это может быть объяснено большей суммой, сроком кредита и большей ответственности к этому. Процент по возвратам кредитов на свадьбу не существенно выше, поэтому можно предположить, что сумма кредита не так велика и оплачивается совместно. Следующая категория по возрастанию - кредиты на образование. Самый большой процент просрочки кредита на авто. Чтобы точно ответить, почему по определенным категориям различается процент невозвратов, необходимо больше данных.  

## 4. Общий вывод

Проверялось четыре гипотезы для кредитного отдела банка:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок? 
Наблюдалось незначительное превышение процентных пунктов невозвратов у группы людей с детьми, чем в группе людей без детей, однако у людей с пятью детьми задолжностей нет. Гипотеза подтвердилась лишь отчасти. 

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
Уровень процентных пунктов невозвратов у людей состоящих в гражданском браке или не женатых самый большой. Овдовевшие или разведенные платят в срок чаще всего. Гипотеза подтвердилась, хотя разница в процентах и небольшая.  

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
У людей со средним доходом уровень процентных пунктов невозвратов самый большой, затем идут люди с низкими доходами, наилучший результат у тех у кого высокий доход. Гипотеза подтвердилась, зависимость есть. 

4. Как разные цели кредита влияют на его возврат в срок?
Кредит на недвижимость имеет самый маленький уровень процентных пунктов невозвратов, самый большой уровень п.п. у кредита на автомобиль. Разница в  процентных пунктах несущественная, данные не показательны. Гипотеза подтвердилась лишь отчасти, отклонения несущественные.

Из данных в анализируемых таблицах нельзя сделать однозначных выводов, разница в п.п. небольшая, результаты не показательны.    